# Implementation of IS-UserBased-Graph
From the paper "Graph-based context-aware collaborative filtering" by Tu Minh Phuong, Do Thi Lien, Nguyen Duy Phuong

Note: The dataset has been preprocessed by removing duplicates

In [1]:
# Step 1: Imports

import pandas as pd # Pandas for importing data
from itertools import product # To calculate the product in item-splitting
import numpy as np # General math stuff
import networkx as nx # For generating graph networks
from scipy.sparse import csr_matrix, lil_matrix, coo_matrix # for sparse matrix representation
from collections import Counter # For summing up a list
from sklearn.model_selection import KFold
import time

In [2]:
full_data = pd.read_csv('../../data/ml100kC/context_ratings_sorted.csv', sep=';')

kf = KFold(n_splits = 10, shuffle = True, random_state = 2)
result = next(kf.split(full_data), None)

train = full_data.iloc[result[0]]
test =  full_data.iloc[result[1]]

# Filter the test set. Used for the 20% withhold data they do in the paper.
#y = test[['userId']]
#test_precision_filter = test[y.replace(y.apply(pd.Series.value_counts)).gt(19).all(1)]

# Calculate how much 20% of the data is for each user in test set
#percentage20 = np.floor(test_precision_filter['userId'].value_counts()*0.20).astype(int)
#test_userIds = test_precision_filter['userId'].unique()
#withheld_data = [] 

#test_precision_filter = test_precision_filter.reset_index()

#for userId in test_userIds:
 #   number_to_withhold = percentage20[userId]
  #  for index, row in test_precision_filter.iterrows():
   #     if row['userId'] == userId:
    #       if number_to_withhold > 0:
     #        withheld_data.append(row)
      #       number_to_withhold -= 1
 
#withheld_dataframe = pd.DataFrame(withheld_data)
#test_precision_filter = test_precision_filter[~test_precision_filter.isin(withheld_dataframe)].dropna()
       
        


#msk = np.random.rand(len(full_data)) < 0.80
#train = full_data[msk] # Training data
#test_data = full_data[~msk] # Test data



## Step 1 (item splitting)
Transform the original multi-dimensional matrix into 2D user x item rating matrix 

In [3]:
# Step 0: Generate all values for Nc1, Nc2 ... Ncn

items_distinct = train.movieId.unique()

# Context dimensions
timeofday_distinct = train.timeofday.unique() # Included
dayofweek_distinct = train.dayofweek.unique() # Included


In [4]:
context_dimensions_product = product(timeofday_distinct, dayofweek_distinct)
T_temp = list(product(context_dimensions_product, items_distinct))
              
T = {k: v for k, v in enumerate(T_temp)}


In [6]:
# Step 3: Generate the two-dimensional matrix of ratings
# This is saved as a sparse matrix, since a lot of the ratings will be unknown
users_distinct = train.userId.unique()
user_ids = {k: v for v, k in enumerate(users_distinct)}

W = lil_matrix((len(users_distinct), len(T)), dtype=np.float32)

# Iterate through ratings and put them in the matrix
progress = 1
start_time = time.time()
train_size = len(train)
for index, row in train.iterrows():
    if progress % 2500 == 0:
        time_spent = time.time() - start_time
        time_per_iter = time_spent/progress
        print(f'{progress} of {train_size}, time: {time_spent/60:.3f} minutes, remaining: {(((train_size - progress)*time_per_iter)/60):.3f} minutes')
    
    itemId = row['movieId']
    userId = row['userId']
    timeofday = row['timeofday']
    dayofweek = row['dayofweek']
    rating = row['rating']
    
    # Find the index for the item 
    idx = list(T.keys())[list(T.values()).index(((timeofday, dayofweek), itemId))]
    
    W[user_ids[userId], idx] = rating
    progress += 1
    
W = csr_matrix(W)

2500 of 90752, time: 0.644 minutes, remaining: 22.750 minutes
5000 of 90752, time: 1.272 minutes, remaining: 21.810 minutes
7500 of 90752, time: 1.907 minutes, remaining: 21.168 minutes
10000 of 90752, time: 2.642 minutes, remaining: 21.333 minutes
12500 of 90752, time: 3.173 minutes, remaining: 19.862 minutes
15000 of 90752, time: 3.885 minutes, remaining: 19.620 minutes
17500 of 90752, time: 4.624 minutes, remaining: 19.356 minutes
20000 of 90752, time: 5.330 minutes, remaining: 18.856 minutes
22500 of 90752, time: 5.973 minutes, remaining: 18.120 minutes
25000 of 90752, time: 6.598 minutes, remaining: 17.352 minutes
27500 of 90752, time: 7.323 minutes, remaining: 16.844 minutes
30000 of 90752, time: 7.984 minutes, remaining: 16.167 minutes
32500 of 90752, time: 8.741 minutes, remaining: 15.667 minutes
35000 of 90752, time: 9.328 minutes, remaining: 14.858 minutes
37500 of 90752, time: 10.163 minutes, remaining: 14.433 minutes
40000 of 90752, time: 10.907 minutes, remaining: 13.838 m

## Step 2: Graph construction
In this step, we'll transform the users_by_T_matrix into a graph to allow us to do graph-based similarity in next step

In [8]:
#edgelist = []
#users_in_graph = set([])

#for user, item in zip(*users_by_T_matrix.nonzero()):
#    rating_value = users_by_T_matrix[user, item]
#    users_in_graph.add("u" + str(user))
#    edgelist.append("u" + str(user) + " i" + str(item) + " " + str(rating_value/5))

#graph = nx.bipartite.edgelist.parse_edgelist(edgelist, data=(('weight', float), ))
#graph.remove_nodes_from(nx.isolates(graph))

## Step 3: Graph-based similarity calculation


In [9]:
# Calculate UZ and W 
# UZ = np.zeros((len(users_distinct), len(items_distinct)), dtype=np.float32)

In [10]:
L = 10 # Amount of jumps, must be an even number

# Generate UZ_L based on the L value
UZ2 = W.dot(np.transpose(W))
UZL = UZ2
# Use jumps to keep track of how many jumps we've made
jumps = 2
while jumps < L:
    UZL = np.dot(UZ2, UZL)
    jumps += 2

### This can also be done using the graph

In [11]:
# Implement graph approach

## Generate recommendations
Now that we have the most similar users, we can find the best items for a given user

In [12]:
def predict(K1, K2, Ua, context, itemid=0):
    
    if Ua not in user_ids:
        print(f'User {Ua} not in trainset: {user_ids}')
        return [], []

    fictional_id = user_ids[Ua]
    
    most_similar_users = []
    for le, ri in zip(UZL.indptr[:-1], UZL.indptr[1:]):
        most_similar_users = UZL.indices[le + np.argpartition(UZL.data[le:ri], -K1)[-K1:]]

    # Find items that the user has rated yet
    user_row = W[fictional_id]
    
    # Recommendations, they'll be the final ones.
    recs = {}

    # itemID, sum rating, rating count
    # k: itemID, v: (sum rating, rating count)
    rating_sum_counter = {}

    for sim_user_row in most_similar_users:
        for row, col in zip(*W[sim_user_row].nonzero()):
            if W[fictional_id, col]:
                continue

            rating = W[sim_user_row, col]
            if col not in rating_sum_counter:
                rating_sum_counter[col] = (rating, 1)
            else:
                current_rating = rating_sum_counter[col]
                new_rating = current_rating[0] + rating
                new_count = current_rating[1] + 1
                rating_sum_counter[col] = (new_rating, new_count)

    # Go through dictionary and sum values
    for k, v in rating_sum_counter.items():
        rating_sum_counter[k] = v[0]/v[1]

    # Sort list by highest values
    counted_recs = Counter(rating_sum_counter)

    # Map back from fictious item to actual item
    item_ids = {v: k for v, k in enumerate(T)}

    filtered_results = []
    for item in counted_recs:
        if T[item][0] == context:
            filtered_results.append(T[item][1])

    filtered_results = filtered_results[:K2]
    
    # Find predicted rating for item
    ## Map from actual item with context to fictional item id
    ## find rating for item if available
    idx = -1
    
    predicted_rating = 0
    for key, value in T.items():
        if value == (context, itemid):
            idx = key
            if key in rating_sum_counter:
                predicted_rating = rating_sum_counter[idx]
                return filtered_results, predicted_rating
            else:
                break

    return filtered_results, predicted_rating

# Calculate hit count

In [17]:
average_precisions = []

def calculate_map(topK_dict, k_val):
    for key in topK_dict:
        score = 0.0
        number_of_hits = 0.0

        for count, element in enumerate(topk_dict[key]):
            if element in relevant_items:
                number_of_hits += 1
                score += number_of_hits / (i + 1)
        
        
        if relevant_items.empty:
            average_precisions.append(0.0)
        elif len(relevant_items) < k_val:
            average_precisions.append(score / len(relevant_items))
        else:
            average_precisions.append(score / k_val)
    
    return (1 / len(test_userIds) * sum(average_precisions))

In [18]:
progress = 1
test_set_size = len(test)
neighbor_values = [10]
K_values = [10]


#relevant_items = [entry for entry in test_precision_filter if test_precision_filter['rating'] > 3]



print('| Neighbors | L | RMSE | K | MAP@K |')
print('|-----------|---|------|---|-------|')
for k_val in K_values:
    for neighbor_val in neighbor_values:
        start_time = time.time()
        neighbor_value = neighbor_val

        actuals = []
        predictions = []
        rmse = 0
        precision = 0
        

        for index, user in test.iterrows():          
            if progress % (test_set_size // 10) == 0:
                time_spent = time.time() - start_time
                time_per_iter = time_spent/progress
                print(f'{progress} of {test_set_size}, time: {time_spent/60:.3f} minutes, remaining: {(((test_set_size - progress)*time_per_iter)/60):.3f} minutes')
            progress += 1

            timeofday = user['timeofday']
            dayofweek = user['dayofweek']

            current_context = (timeofday, dayofweek)

            item = user['movieId']
            user_id = user['userId']
            rating = user['rating']

            topK, prediction = predict(K1=neighbor_value, K2=k_val, Ua=user_id, context=current_context, itemid=item)

            if prediction != 0:
                actuals.append(rating)
                predictions.append(prediction)
            
        if len(predictions) > 0:
            rmse = np.sqrt(np.mean((np.array(predictions)-np.array(actuals))**2))
        

        print(f'| {neighbor_value} | {L} | {rmse:.3f} | mapk | {k_val}')
        progress = 1
        start_time = time.time()

| Neighbors | L | RMSE | K | MAP@K |
|-----------|---|------|---|-------|
554 of 5541, time: 6.978 minutes, remaining: 62.818 minutes
1108 of 5541, time: 13.668 minutes, remaining: 54.685 minutes
1662 of 5541, time: 20.236 minutes, remaining: 47.229 minutes
2216 of 5541, time: 26.746 minutes, remaining: 40.132 minutes
2770 of 5541, time: 33.336 minutes, remaining: 33.348 minutes
3324 of 5541, time: 39.786 minutes, remaining: 26.536 minutes
3878 of 5541, time: 46.088 minutes, remaining: 19.764 minutes
4432 of 5541, time: 52.635 minutes, remaining: 13.171 minutes
4986 of 5541, time: 58.839 minutes, remaining: 6.550 minutes
5540 of 5541, time: 65.206 minutes, remaining: 0.012 minutes
| 10 | 10 | 1.077 | 0.000 | 10


# Results


# U1 split
| Neighbors | L  | RMSE   |
|-----------|----|--------|
| 20        | 10 | 1\.168 |
| 10        | 8  | 1\.169 |
| 20        | 6  | 1\.172 |
| 10        | 12 | 1\.173 |
| 10        | 10 | 1\.173 |
| 10        | 6  | 1\.200 |
| 5         | 8  | 1\.216 |
| 5         | 10 | 1\.216 |
| 5         | 6  | 1\.231 |
| 3         | 8  | 1\.279 |
| 3         | 10 | 1\.279 |
| 3         | 12 | 1\.279 |
| 20        | 4  | 1\.294 |
| 10        | 4  | 1\.323 |
| 3         | 6  | 1\.337 |
| 3         | 4  | 1\.361 |
| 5         | 4  | 1\.365 |


# Missing:
U1: L4, N20

In [ ]:
#How to get

#relevant_items = withheld_dataframe.loc[(withheld_dataframe['rating'] > 3) & (withheld_dataframe['userId'] == key)]
        
#intersect_relevant_topk = relevant_items[relevant_items.index.isin(topK_dict[key])]